In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import KFold
import h5py
from six.moves import cPickle as pickle
from helper import standardization, normalize_train_test
from model import model_FCN9_base

Using TensorFlow backend.


## Load data(original scale)

In [2]:
with open('../Data/Exp14_128/rigid-registration/MRICT_patch_24_24.pickle', "rb") as f:
    data_patch = pickle.load(f)

MR_patch = data_patch['MRI'].astype('float32')
CT_patch = data_patch['CT'].astype('float32')
num_patients = data_patch['num_patients']
del data_patch

In [3]:
# batch, spatial_dim1,spatial_dim2, spatial_dim3, channels
# data_format = "channels_last" default
MR_patch.shape, CT_patch.shape

((51030, 24, 24, 24), (51030, 24, 24, 24))

## 7 patients for training, and 7 for testing

In [4]:
patches_per_patient = MR_patch.shape[0]/num_patients
# training and testing data
train_MR = MR_patch[:-7*patches_per_patient]
train_CT = CT_patch[:-7*patches_per_patient]

test_MR = MR_patch[-7*patches_per_patient:]
test_CT = CT_patch[-7*patches_per_patient:]

In [5]:
train_MR, test_MR = normalize_train_test(train_MR, test_MR)

In [6]:
train_CT, test_CT = normalize_train_test(train_CT, test_CT)

In [7]:
train_MR.mean(),train_MR.std()
train_CT.mean(),train_CT.std()
test_MR.mean(),test_MR.std()
test_CT.mean(),test_CT.std()

(1.5173628e-05, 1.0000004)

(2.7066622e-05, 1.0000048)

(0.25648129, 1.0552354)

(-0.014244468, 1.1235565)

## Build model

In [6]:
#hyperparameters
batch_size = 64
epochs = 100
lr = 1e-06

In [7]:
model = model_FCN9_base(lr) #he initialization

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding3d_1 (ZeroPaddin (None, 26, 26, 26, 1)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 24, 24, 24, 32)    864       
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 24, 32)    128       
_________________________________________________________________
activation_1 (Activation)    (None, 24, 24, 24, 32)    0         
_________________________________________________________________
zero_padding3d_2 (ZeroPaddin (None, 26, 26, 26, 32)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 24, 24, 24, 32)    27648     
_________________________________________________________________
batch_normalization_2 (Batch (None, 24, 24, 24, 32)    128       
__________

In [ ]:
#complile model
nadam = optimizers.Nadam(lr=LearningRate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)

model.compile(optimizer=nadam,
              loss=losses.mean_squared_error,
              metrics= [metrics.mean_absolute_error])

In [10]:
# only train for one epoch as an example
history = model.fit(train_MR, train_CT, batch_size = batch_size, epochs = 1,
                    verbose =1, validation_data = (test_MR,test_CT))

Train on 25515 samples, validate on 25515 samples
Epoch 1/1
25515/25515 [==============================] - 484s - loss: 2.9665 - mean_absolute_error: 1.2647 - val_loss: 2.8954 - val_mean_absolute_error: 1.2351

## Save model

In [ ]:
model_dir = '../Models/model_0501/'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

In [ ]:
model.save(os.path.join(model_dir, '3DFCN9_TrainFirst7patients_model.hdf5' ))
print("Saved model to disk")

historyfile = os.path.join(model_dir, '3DFCN9_TrainFirst7patients_history.csv')
pd.DataFrame(history.history).to_csv(historyfile)
print("Saved history to disk")

del model

In [8]:
kfold = KFold(n_splits = 2, shuffle = False, random_state = None)

In [11]:
train_index, cv_index = kfold.split(MR_patch, CT_patch)